In [58]:
import pandas as pd
import gensim.downloader as api
import numpy as np
import re
import gensim.downloader as api
from gensim.models import Word2Vec, FastText

# Load GloVe embeddings from gensim's online repository
glove_word_vectors = api.load('glove-wiki-gigaword-100')

# Load Word2Vec embeddings from gensim's online repository
word2vec_model = api.load('word2vec-google-news-300')

# Load FastText embeddings from gensim's online repository
fasttext_model = api.load('fasttext-wiki-news-subwords-300')

#model = api.load('glove-wiki-gigaword-100')  # download the model and return as object ready for use

[==================================================] 100.0% 128.1/128.1MB downloaded


In [174]:

file_path = "mycards.json"
df = pd.read_json(file_path)


In [175]:
df = df[df.lang == 'en']
df = df[df.layout != 'token']
df = df[~((df.name == 'Plains') | (df.name == 'Island') | (df.name == 'Swamp') | (df.name == 'Mountain') | (df.name == 'Forest'))]
df = df.drop_duplicates(subset=['name'])




In [176]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 30332 entries, 0 to 93484
Data columns (total 85 columns):
 #   Column               Non-Null Count  Dtype         
---  ------               --------------  -----         
 0   object               30332 non-null  object        
 1   id                   30332 non-null  object        
 2   oracle_id            30303 non-null  object        
 3   multiverse_ids       30332 non-null  object        
 4   mtgo_id              18578 non-null  float64       
 5   mtgo_foil_id         10875 non-null  float64       
 6   tcgplayer_id         25872 non-null  float64       
 7   cardmarket_id        23891 non-null  float64       
 8   name                 30332 non-null  object        
 9   lang                 30332 non-null  object        
 10  released_at          30332 non-null  datetime64[ns]
 11  uri                  30332 non-null  object        
 12  scryfall_uri         30332 non-null  object        
 13  layout               30332 non-

In [177]:
cols_to_drop = np.array(df.columns[[0, 2, 3, 4, 5, 6, 7, 9, 10, 11, 12, 13, 14, 15, 16, 27, 28, 29, 30, 31, 32, 33] + list(range(34, 85))])

# Drop the columns
df.drop(columns=cols_to_drop, inplace=True)

In [178]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 30332 entries, 0 to 93484
Data columns (total 12 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   id              30332 non-null  object 
 1   name            30332 non-null  object 
 2   mana_cost       28420 non-null  object 
 3   cmc             30303 non-null  float64
 4   type_line       30303 non-null  object 
 5   oracle_text     28169 non-null  object 
 6   power           15317 non-null  object 
 7   toughness       15317 non-null  object 
 8   colors          28420 non-null  object 
 9   color_identity  30332 non-null  object 
 10  keywords        30332 non-null  object 
 11  legalities      30332 non-null  object 
dtypes: float64(1), object(11)
memory usage: 3.0+ MB


In [186]:
df[df.power.isnull()]


,id,name,mana_cost,cmc,type_line,oracle_text,power,toughness,colors,color_identity,keywords,legalities
4,00012bd8-ed68-4978-a22d-f450c8a6e048,Web,{G},1.0,Enchantment — Aura,Enchant creature (Target a creature as you cast this. This card enters the battlefield attached to that creature.)\nEnchanted creature gets +0/+2 and has reach. (It can block creatures with flying.),NaN,NaN,[G],[G],[Enchant],"{'standard': 'not_legal', 'future': 'not_legal', 'historic': 'not_legal', 'timeless': 'not_legal', 'gladiator': 'not_legal', 'pioneer': 'not_legal', 'explorer': 'not_legal', 'modern': 'legal', 'legacy': 'legal', 'pauper': 'not_legal', 'vintage': 'legal', 'penny': 'legal', 'commander': 'legal', 'oathbreaker': 'legal', 'standardbrawl': 'not_legal', 'brawl': 'not_legal', 'alchemy': 'not_legal', 'paupercommander': 'not_legal', 'duel': 'legal', 'oldschool': 'legal', 'premodern': 'legal', 'predh': 'legal'}"
5,0001c639-8bd0-426f-89cb-4ca61f3cc054,Surge of Brilliance,{1}{U},2.0,Instant,"Paradox — Draw a card for each spell you've cast this turn from anywhere other than your hand.\nForetell {1}{U} (During your turn, you may pay {2} and exile this card from your hand face down. Cast it on a later turn for its foretell cost.)",NaN,NaN,[U],[U],"[Paradox, Foretell]","{'standard': 'not_legal', 'future': 'not_legal', 'historic': 'not_legal', 'timeless': 'not_legal', 'gladiator': 'not_legal', 'pioneer': 'not_legal', 'explorer': 'not_legal', 'modern': 'not_legal', 'legacy': 'legal', 'pauper': 'not_legal', 'vintage': 'legal', 'penny': 'not_legal', 'commander': 'legal', 'oathbreaker': 'legal', 'standardbrawl': 'not_legal', 'brawl': 'not_legal', 'alchemy': 'not_legal', 'paupercommander': 'not_legal', 'duel': 'legal', 'oldschool': 'not_legal', 'premodern': 'not_legal', 'predh': 'not_legal'}"
8,00020b05-ecb9-4603-8cc1-8cfa7a14befc,Wildcall,{X}{G}{G},2.0,Sorcery,"Manifest the top card of your library, then put X +1/+1 counters on it. (To manifest a card, put it onto the battlefield face down as a 2/2 creature. Turn it face up any time for its mana cost if it's a creature card.)",NaN,NaN,[G],[G],[Manifest],"{'standard': 'not_legal', 'future': 'not_legal', 'historic': 'not_legal', 'timeless': 'not_legal', 'gladiator': 'not_legal', 'pioneer': 'legal', 'explorer': 'not_legal', 'modern': 'legal', 'legacy': 'legal', 'pauper': 'not_legal', 'vintage': 'legal', 'penny': 'legal', 'commander': 'legal', 'oathbreaker': 'legal', 'standardbrawl': 'not_legal', 'brawl': 'not_legal', 'alchemy': 'not_legal', 'paupercommander': 'not_legal', 'duel': 'legal', 'oldschool': 'not_legal', 'premodern': 'not_legal', 'predh': 'not_legal'}"
17,0005968a-8708-441b-b9a1-9373aeb8114d,Mulch,{1}{G},2.0,Sorcery,Reveal the top four cards of your library. Put all land cards revealed this way into your hand and the rest into your graveyard.,NaN,NaN,[G],[G],[],"{'standard': 'legal', 'future': 'legal', 'historic': 'legal', 'timeless': 'legal', 'gladiator': 'legal', 'pioneer': 'legal', 'explorer': 'legal', 'modern': 'legal', 'legacy': 'legal', 'pauper': 'legal', 'vintage': 'legal', 'penny': 'legal', 'commander': 'legal', 'oathbreaker': 'legal', 'standardbrawl': 'legal', 'brawl': 'legal', 'alchemy': 'not_legal', 'paupercommander': 'legal', 'duel': 'legal', 'oldschool': 'not_legal', 'premodern': 'legal', 'predh': 'legal'}"
22,000809e6-2dd5-41cc-a316-3edb4e40eb58,Siren's Call,{U},1.0,Instant,"Cast this spell only during an opponent's turn, before attackers are declared.\nCreatures the active player controls attack this turn if able.\nAt the beginning of the next end step, destroy all non-Wall creatures that player controls that didn't attack this turn. Ignore this effect for each creature the player didn't control continuously since the beginning of the turn.",NaN,NaN,[U],[U],[],"{'standard': 'not_legal', 'future': 'not_legal', 'historic': 'not_legal', 'timeless': 'not_legal', 'gladiator': 'not_legal', 'pioneer': 'not_legal', 'explorer': 'not_legal', 'modern': 'not_legal', 'legacy': 'legal', 'pauper': '

In [205]:

# Function to preprocess text
def preprocess_text(text):
    if isinstance(text, float):
        text = str(text)
    text = text.lower()
    text = text.replace("\n", " ")
    text = re.sub(r'{[^}]+}', 'symbol', text)
    text = re.sub(r'[^\w\s]', '', text)
    return text.split()

# Function to obtain embeddings using GloVe
def get_glove_embedding(tokens):
    embeddings = []
    for token in tokens:
        try:
            embeddings.append(glove_word_vectors[token])
        except KeyError:
            # Handle out-of-vocabulary tokens
            pass
    if embeddings:
        return np.mean(embeddings, axis=0)
    else:
        return np.zeros(glove_word_vectors.vector_size)

# Function to obtain embeddings using Word2Vec
def get_word2vec_embedding(tokens):
    embeddings = []
    for token in tokens:
        if token in word2vec_model:
            embeddings.append(word2vec_model[token])
    if embeddings:
        return np.mean(embeddings, axis=0)
    else:
        return np.zeros(word2vec_model.vector_size)

# Function to obtain embeddings using FastText
def get_fasttext_embedding(tokens):
    embeddings = []
    for token in tokens:
        if token in fasttext_model:
            embeddings.append(fasttext_model[token])
    if embeddings:
        return np.mean(embeddings, axis=0)
    else:
        return np.zeros(fasttext_model.vector_size)

# Apply preprocessing and get embeddings for each embedding type
df['oracle_text_tokens'] = df['oracle_text'].apply(preprocess_text)
df['glove_embedding'] = df['oracle_text_tokens'].apply(get_glove_embedding)
df['word2vec_embedding'] = df['oracle_text_tokens'].apply(get_word2vec_embedding)
df['fasttext_embedding'] = df['oracle_text_tokens'].apply(get_fasttext_embedding)

In [206]:
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler
from sklearn.neighbors import NearestNeighbors
# Encode the 'mana_cost' feature
label_encoder = LabelEncoder()
df['mana_cost_encoded'] = label_encoder.fit_transform(df['mana_cost'])

# Concatenate features including vectorized text columns and other features
X = np.concatenate([np.array(df['glove_embedding'].tolist()), np.array(df['mana_cost_encoded'].tolist()).reshape(-1, 1)], axis=1)

# Normalize features
scaler = StandardScaler()
X_normalized = scaler.fit_transform(X)

# Train KNN model
knn = NearestNeighbors(n_neighbors=5, algorithm='auto')  # You can adjust the number of neighbors (k) as needed
knn.fit(X_normalized)

NearestNeighbors()

In [207]:
df.mana_cost_encoded[0]

577

In [208]:
def get_features_for_card(card_name_or_id):
    # Assuming 'card_name_or_id' is the name or ID of the card
    # Here you would extract the features for the given card from your DataFrame
    # For simplicity, let's assume 'card_features' is a vector containing the features for the card
    card_features = df.loc[df['name'] == card_name_or_id, ['glove_embedding', 'mana_cost_encoded']].values[0]
    glove_embedding = card_features[0]  # Extract glove_embedding
    mana_cost_encoded = np.array([card_features[1]])  # Extract mana_cost_encoded and ensure it's a 1D array
    return np.concatenate((glove_embedding, mana_cost_encoded))

input_card_name_or_id = "Fury Sliver"  # Provide the name or ID of the input card

# Extract features for the input card
input_card_features = get_features_for_card(input_card_name_or_id)

# Normalize features for the input card
input_card_features_normalized = scaler.transform(np.array(input_card_features).reshape(1, -1))

# Find most similar cards to the input card
distances, indices = knn.kneighbors(input_card_features_normalized)

# Output most similar cards
most_similar_cards = df.iloc[indices[0]]



In [209]:
len(most_similar_cards['name'])

5

In [211]:
most_similar_cards['name']

0            Fury Sliver
74996    Spitting Sliver
88280       Talon Sliver
2955       Lancer Sliver
9256     Rage Reflection
Name: name, dtype: object

In [203]:
# Convert 'colors' and 'color_identity' columns to numeric representation
X['colors'] = X['colors'].apply(lambda x: len(x) if isinstance(x, str) else np.nan).astype(float)
X['color_identity'] = X['color_identity'].apply(lambda x: len(x) if isinstance(x, str) else np.nan).astype(float)

# Convert X dataframe to numpy array
X_array = np.array(X)

# Normalize features
scaler = StandardScaler()
X_normalized = scaler.fit_transform(X_array)


C:\Users\dcw1981\AppData\Local\Temp\ipykernel_26000\429743573.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X['colors'] = X['colors'].apply(lambda x: len(x) if isinstance(x, str) else np.nan).astype(float)
C:\Users\dcw1981\AppData\Local\Temp\ipykernel_26000\429743573.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  X['color_identity'] = X['color_identity'].apply(lambda x: len(x) if isinstance(x, str) else np.nan).astype(float)


ValueError: setting an array element with a sequence.

In [197]:

# Normalize features
scaler = StandardScaler()
X_normalized = scaler.fit_transform(X.values.reshape(-1, 1))


ValueError: setting an array element with a sequence.

In [ ]:

# Train KNN model
knn = NearestNeighbors(n_neighbors=5, algorithm='auto')  # You can adjust the number of neighbors (k) as needed
knn.fit(X_normalized)

In [110]:
from sklearn.impute import SimpleImputer
# Replace NaN values with mean
imputer = SimpleImputer(strategy='mean')
X_imputed = imputer.fit_transform(X_normalized)

# Train KNN model with imputed data
knn.fit(X_imputed)


ValueError: Input X contains NaN.
NearestNeighbors does not accept missing values encoded as NaN natively. For supervised learning, you might want to consider sklearn.ensemble.HistGradientBoostingClassifier and Regressor which accept missing values encoded as NaNs natively. Alternatively, it is possible to preprocess the data, for instance by using an imputer transformer in a pipeline or drop samples with missing values. See https://scikit-learn.org/stable/modules/impute.html You can find a list of all estimators that handle NaN values at the following page: https://scikit-learn.org/stable/modules/impute.html#estimators-that-handle-nan-values

In [ ]:
# Function to get features for a given card name or ID
def get_features_for_card(card_name_or_id):
    # Assuming 'card_name_or_id' is the name or ID of the card
    # Here you would extract the features for the given card from your DataFrame
    # For simplicity, let's assume 'card_features' is a vector containing the features for the card
    card_features = df.loc[df['name'] == card_name_or_id, ['glove_embedding', 'cmc']].values[0]
    return card_features

# Example: Find most similar cards to a new card
input_card_name_or_id = "Fury Sliver"  # Provide the name or ID of the input card
input_card_features = get_features_for_card(input_card_name_or_id)
input_card_features_normalized = scaler.transform(input_card_features.reshape(1, -1))
distances, indices = knn.kneighbors(input_card_features_normalized)

# Output most similar cards
most_similar_cards = df.iloc[indices[0]]
print(most_similar_cards)
